# Nebraska Cornhuskers vs. Ohio State Buckeyes Game: 9/29

I used Tweepy to capture a set of streaming tweets about the Husker game on September 28th, using the keywords 'husker,''huskers,''cornhuskers,''gbr,''ohio state,''buckeyes,''gobucks,''OSUvsNEB.' I got the following information related to each Tweet: user name, Tweet ID, the time posted, the user's bio-specified location, Tweet text, the tweet's geo coordinates from which it was sent, whether the Tweet was a Retweet, and whether or not the Tweet was a quote tweet. The code to get Tweets is also available in my Git repository.

In [238]:
#importing libraries

import pandas as pd
import re
from datetime import datetime, timedelta
from textblob import TextBlob

# getting the data 
tweets = pd.read_csv("Documents/Husker Project/gameTweets.csv")

#checking the info about the tweets

print(tweets.columns)
print(tweets.shape)

Index(['Username', 'Tweet ID', 'Time', 'User Location', 'Text',
       'Tweet Geo Coordinates', 'Is Retweet', 'Is Quote Tweet'],
      dtype='object')
(46439, 8)


As you can see above, I ended up getting over 46,000 Tweets.

In [239]:
# a bit more information about the data I collected

tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46439 entries, 0 to 46438
Data columns (total 8 columns):
Username                 46439 non-null object
Tweet ID                 46439 non-null int64
Time                     46439 non-null object
User Location            33978 non-null object
Text                     46439 non-null object
Tweet Geo Coordinates    35 non-null object
Is Retweet               46439 non-null bool
Is Quote Tweet           46439 non-null bool
dtypes: bool(2), int64(1), object(5)
memory usage: 2.2+ MB


Using some basic descriptions, I can get who Tweeted the most about the game (or, at least, who Tweeted the most using my keywords):

In [240]:
print(tweets['Username'].value_counts(dropna=False))


CBJBucksCrew614    87
roymdavisll        81
Scarletjersey      70
Azcroat            65
eric_delaluz       65
Dukeeeeee_         56
BonesHusker        56
buckeyeplanet      54
BrianDa83878772    51
_Heavyy            50
Romedawwwg         48
dark____master     48
Buckeyefanohio7    43
JuJu_Da_Gamer      42
Dan_Hope           42
TNTABDI            42
CodySlam           40
GEWilliams17       40
kelitos_way        39
KJS_football_78    39
4MEDLEN            39
Alaina92871202     37
_foodoverlove      37
ShalashMuh         37
jakedasnake27      36
russhein43         36
OWHbigred          34
steezmcgavin       34
klovex12           33
dismisstrump       33
                   ..
Reeed210000         1
charles90542553     1
sampasser           1
J_Labbs             1
sarahdactyl93       1
BigDickerEnergy     1
Nathan_Wear         1
gd04051973          1
jason__hernan       1
THEchrisHOKE        1
WVUMajor9           1
joshstevenson23     1
aijahogan           1
Maximizealot        1
Live4Gr8ne

And what time most Tweets are posted. This isn't very useful, as it's on a second-by-second basis (and also in UTC).

In [241]:
print(tweets['Time'].value_counts(dropna=False))

2019-09-29 00:42:32    22
2019-09-29 00:42:09    19
2019-09-29 00:39:58    19
2019-09-29 01:11:20    18
2019-09-29 00:43:33    17
2019-09-29 00:42:42    17
2019-09-29 00:42:37    17
2019-09-29 01:11:31    17
2019-09-29 01:11:26    16
2019-09-29 00:42:16    16
2019-09-29 01:11:11    16
2019-09-29 00:54:19    16
2019-09-29 01:13:27    16
2019-09-29 00:42:49    16
2019-09-29 00:54:20    16
2019-09-29 00:43:22    16
2019-09-29 00:12:04    16
2019-09-29 00:54:43    16
2019-09-29 00:43:39    16
2019-09-29 00:43:04    15
2019-09-29 00:54:26    15
2019-09-29 01:12:14    15
2019-09-29 00:42:53    15
2019-09-29 01:05:58    15
2019-09-29 01:12:32    15
2019-09-29 00:43:08    15
2019-09-29 00:42:51    15
2019-09-29 00:43:03    15
2019-09-29 00:45:23    15
2019-09-29 00:41:38    15
                       ..
2019-09-29 01:54:14     1
2019-09-28 23:59:13     1
2019-09-29 02:26:42     1
2019-09-29 00:32:35     1
2019-09-29 00:15:15     1
2019-09-29 02:10:37     1
2019-09-29 02:10:49     1
2019-09-29 0

Now, for user location! Given how rabid Husker fans can be, I expected more Nebraska locations, but Columbus, OH came out on top. This is a user-generated field (as noted by some of the unusual values down at the bottom, like "McDonalds," "Lost Continent," and a Bible verse), so it'll probably need some cleaning.

In [242]:
print(tweets['User Location'].value_counts())

Columbus, OH                      1749
Ohio, USA                          909
United States                      733
Lincoln, NE                        686
Omaha, NE                          650
Nebraska, USA                      470
Ohio                               455
Cleveland, OH                      410
Columbus, Ohio                     381
Nebraska                           310
Atlanta, GA                        231
Florida, USA                       203
Cincinnati, OH                     194
Chicago, IL                        189
Dallas, TX                         167
Texas, USA                         151
Los Angeles, CA                    151
Lincoln, Nebraska                  150
Las Vegas, NV                      133
Houston, TX                        120
USA                                120
Charlotte, NC                      112
Washington, DC                     111
Georgia, USA                       106
Austin, TX                         100
Dayton, OH               

Geo coordinates tell where the Tweet was sent from. Unfortunately, very few users have this option turned on, so only a few (35) Tweets showed up.

In [243]:
print(tweets['Tweet Geo Coordinates'].value_counts(dropna=False))

NaN                                                               46404
{'type': 'Point', 'coordinates': [-83.01441447, 40.00142341]}         3
{'type': 'Point', 'coordinates': [-87.85535431, 42.287323]}           3
{'type': 'Point', 'coordinates': [-96.70594765, 40.82043599]}         2
{'type': 'Point', 'coordinates': [-118.02102287, 33.92233478]}        2
{'type': 'Point', 'coordinates': [-118.32929, 34.09996]}              1
{'type': 'Point', 'coordinates': [-82.9781698, 40.0630404]}           1
{'type': 'Point', 'coordinates': [-96.70600897, 40.82057983]}         1
{'type': 'Point', 'coordinates': [-83.5401, 41.6503]}                 1
{'type': 'Point', 'coordinates': [-115.16976497, 36.12164075]}        1
{'type': 'Point', 'coordinates': [-86.22539, 39.8525]}                1
{'type': 'Point', 'coordinates': [-82.93154, 40.12577]}               1
{'type': 'Point', 'coordinates': [-83.6289, 41.55671]}                1
{'type': 'Point', 'coordinates': [-94.58214819, 39.097229]}     

And, lastly, whether or not a Tweet is a quote Tweet or a Retweet. This could be nice to sort on when I just want to analyze unique Tweets, and also for satisfying my curiosity if more people Tweeted or Retweeted game Tweets. It looks like most Tweets weren't Retweets, but they were comparable in volume.

In [244]:
print(tweets['Is Retweet'].value_counts(dropna=False))
print(tweets['Is Quote Tweet'].value_counts(dropna=False))

False    24925
True     21514
Name: Is Retweet, dtype: int64
False    40997
True      5442
Name: Is Quote Tweet, dtype: int64


# Cleaning!

Now that I've got a feel for my data, I'm going to do some cleaning.

First, cleaning the Tweets. I'm going to make everything lower case, and consider only non-Retweets, loading these into a new dataframe.

In [245]:
nonRetweets = tweets.loc[tweets['Is Retweet'] == False].copy() # was getting a SettingWithCopy warning, so making sure that I specified to make a new data set

#making everything lower case

nonRetweets['Text'] = nonRetweets['Text'].str.lower()

nonRetweets.shape #and double checking that I get 24925 values...

(24925, 8)

Now, I need to remove punctuation besides hashtags.

In [246]:
#removing urls first

h = lambda x: re.sub(r"http\S+", "", x)
nonRetweets['Text'] = nonRetweets['Text'].apply(h)

#then getting rid of other punctuation

nonRetweets['Text'] = nonRetweets['Text'].str.replace("[^a-zA-Z#]", " ") #replacing everything that's not a letter or a hashtag with a space instead
nonRetweets['Text'].head() #and checking to see if the data looks good

0     buckeyes at cornhuskers  #osuvsneb  here we g...
1                        ohio state over    team total
2                            go big red  #huskers #gbr
3    if he s  good at distracting he could run for ...
5    last time ohio state played in lincoln was the...
Name: Text, dtype: object

I want to graph sentiment analysis for the Tweets as well, and since this is done using the WordBlob package on strings and not lists of words, I'm going to add a new column and calculate the sentiment score for each individual Tweet now.

In [247]:
# Making a text blob for each tweet, following a great tutorial from earthdatascience.org

sentiment_objects = [TextBlob(tweet) for tweet in nonRetweets['Text']]

#then making a list of polarity values
sentiment_values = [tweet.sentiment.polarity for tweet in sentiment_objects]

In [248]:
#and attaching this list as a new column in the dataframe

nonRetweets['Sentiment'] = sentiment_values

nonRetweets.head(10)

,Username,Tweet ID,Time,User Location,Text,Tweet Geo Coordinates,Is Retweet,Is Quote Tweet,Sentiment
0,NANxox2_WOKE,1178090726854418433,2019-09-28 23:34:51,United States,buckeyes at cornhuskers #osuvsneb here we g...,NaN,False,False,0.00
1,WinnersStrictly,1178090727709872128,2019-09-28 23:34:52,"Oakland, CA",ohio state over team total,NaN,False,False,0.00
2,LucasLee45,1178090728418725889,2019-09-28 23:34:52,The GREAT State of Colorado.,go big red #huskers #gbr,NaN,False,False,0.00
3,Kaine7061,1178090728674791424,2019-09-28 23:34:52,NaN,if he s good at distracting he could run for ...,NaN,False,True,0.70
5,dpm917,1178090731753349122,2019-09-28 23:34:52,NaN,last time ohio state played in lincoln was the...,NaN,False,False,-0.06
6,aderetzi_,1178090733003083777,2019-09-28 23:34:53,NaN,i m cryin,NaN,False,True,0.00
7,MikeSpector01,1178090743547727872,2019-09-28 23:34:55,"New Jersey, USA",when vegas casino heads say they need nebraska...,NaN,False,False,0.00
8,PhilBergmanTV,1178090742197166080,2019-09-28 23:34:55,"Omaha, NE",brock osweiler at today s #huskers game,NaN,False,False,-0.40
9,Prodby2LoKi,1178090747209359361,2019-09-28 23:34:56,"Norfside, Huntsville, AL",shewrap ohio state when they win the aldi kro...,NaN,False,False,0.80
11,DownWithDamon,1178090757892255744,2019-09-28 23:34:59,"Cleveland, OH",let s go buckeyes,NaN,False,False,0.00


Then, splitting Tweets into individual words so that we can see things like word counts and do sentiment analysis

In [249]:
f = lambda x: x.split() #making a lambda function rather than defining something big for one action
nonRetweets['Text'] = nonRetweets['Text'].apply(f)

#checking to be sure it did what I wanted

nonRetweets['Text'].head()

0    [buckeyes, at, cornhuskers, #osuvsneb, here, w...
1                     [ohio, state, over, team, total]
2                       [go, big, red, #huskers, #gbr]
3    [if, he, s, good, at, distracting, he, could, ...
5    [last, time, ohio, state, played, in, lincoln,...
Name: Text, dtype: object

Okay, that's it so far for the Tweets themselves. Another problem area? The user-reported locations. We can see from the most common locations that they are commonly in the formats of "City, Two Letter State," "State, Country," "City, Full State," or simply "State." I'm going to split the location field into whitespace-separated words, and then check these against a list of state names and state abbreviations

In [250]:
# make the 'User Location' field nicer to analyze

nonRetweets['User Location'] = nonRetweets['User Location'].str.replace("[^a-zA-Z]", " ") #replacing everything that's not a letter with a space instead
nonRetweets['User Location'] = nonRetweets['User Location'].str.lower() #lower case
nonRetweets['User Location'] = nonRetweets['User Location'].fillna("") #getting rid of NaN values

# using my previous little lambda function to split

nonRetweets['User Location'] = nonRetweets['User Location'].apply(f)
nonRetweets['User Location'].head()


0                     [united, states]
1                        [oakland, ca]
2    [the, great, state, of, colorado]
3                                   []
5                                   []
Name: User Location, dtype: object

In [251]:
# list of states
states = ["alabama","alaska","arizona","arkansas","california","colorado",
  "connecticut","delaware","florida","georgia","hawaii","idaho","illinois",
  "indiana","iowa","kansas","kentucky","louisiana","maine","maryland",
  "massachusetts","michigan","minnesota","mississippi","missouri","montana",
  "nebraska","nevada","new hampshire","new jersey","new mexico","new york",
  "north carolina","north dakota","ohio","oklahoma","oregon","pennsylvania",
  "rhode island","south carolina","south dakota","tennessee","texas","utah",
  "vermont","virginia","washington","west virginia","wisconsin","wyoming"]

# list of abbreviations

stateAbbreviations =  ["al", "ak", "az", "ar", "ca", "co", "ct", "dc", "de", "fl", "ga", 
          "hi", "id", "il", "in", "ia", "ks", "ky", "la", "me", "md", 
          "ma", "mi", "mn", "ms", "mo", "mt", "ne", "nv", "nh", "nj", 
          "nm", "ny", "nc", "nd", "oh", "ok", "or", "pa", "ri", "sc", 
          "sd", "tn", "tx", "ut", "vt", "va", "wa", "wv", "wi", "wy"]

# dictionary for transitioning between abbreviations and states

abbrevToState = {
        'ak': 'alaska',
        'al': 'alabama',
        'ar': 'arkansas',
        'as': 'american Samoa',
        'az': 'arizona',
        'ca': 'california',
        'co': 'colorado',
        'ct': 'connecticut',
        'dc': 'district of columbia',
        'de': 'delaware',
        'fl': 'florida',
        'ga': 'georgia',
        'hi': 'hawaii',
        'ia': 'iowa',
        'id': 'idaho',
        'il': 'illinois',
        'in': 'indiana',
        'ks': 'kansas',
        'ky': 'kentucky',
        'la': 'louisiana',
        'ma': 'massachusetts',
        'md': 'maryland',
        'me': 'maine',
        'mi': 'michigan',
        'mn': 'minnesota',
        'mo': 'missouri',
        'ms': 'mississippi',
        'mt': 'montana',
        'nc': 'north carolina',
        'nd': 'north dakota',
        'ne': 'nebraska',
        'nh': 'new hampshire',
        'nj': 'new jersey',
        'nm': 'new mexico',
        'nv': 'nevada',
        'ny': 'new york',
        'oh': 'ohio',
        'ok': 'oklahoma',
        'or': 'oregon',
        'pa': 'pennsylvania',
        'ri': 'rhode island',
        'sc': 'south carolina',
        'sd': 'south dakota',
        'tn': 'tennessee',
        'tx': 'texas',
        'ut': 'utah',
        'va': 'virginia',
        'vt': 'vermont',
        'wa': 'washington',
        'wi': 'wisconsin',
        'wv': 'west virginia',
        'wy': 'wyoming'
}


# checking if anything in the user location matches a state abbreviation or a full state name
def stateChecker(list1, list2, list3, dict1):
    if any(item in list1 for item in list2): # first checking the first list against the second
        if len(set(list1)&set(list2)) == 1: # making sure there aren't multiple states listed
            list1 = list(set(list1)&set(list2)) # getting only the value that matches (the abbreviation)
            list1 = list1[0] # and getting only the value itself
            list1 = dict1[list1] #then, setting the value of the user location to the corresponding state
            return True
        else:
            return False
    elif any (item in list1 for item in list3):
        if len(set(list1)&set(list3)) == 1:
            list1 = list(set(list1)&set(list3)) # getting only the value that matches (the state name)
            list1 = list1[0] # and getting only the value itself
            return True
        else:
            return False
    else:
        return False

# then going through the user location field and applying it to each one

nonRetweets['User Location'].apply(stateChecker, args = (stateAbbreviations,states,abbrevToState))

0        False
1         True
2         True
3        False
5        False
6        False
7        False
8         True
9         True
11        True
12       False
14        True
15        True
16        True
17       False
18       False
19        True
21       False
22       False
23       False
25        True
26       False
28       False
29       False
30       False
31       False
33       False
34       False
35        True
36       False
         ...  
46354     True
46358     True
46359     True
46360    False
46361     True
46365    False
46366    False
46368    False
46371     True
46373    False
46382     True
46389    False
46391    False
46395     True
46396     True
46400     True
46404     True
46406    False
46408     True
46410    False
46418    False
46422     True
46423     True
46424     True
46425    False
46428    False
46431    False
46433    False
46434    False
46435    False
Name: User Location, Length: 24925, dtype: bool

In [252]:
# then, getting my final tweet set of tweets only that have an identifiable state location
finalTweetSet = nonRetweets.loc[nonRetweets['User Location'].apply(stateChecker, args = (stateAbbreviations,states,abbrevToState))].copy()

finalTweetSet.head()

,Username,Tweet ID,Time,User Location,Text,Tweet Geo Coordinates,Is Retweet,Is Quote Tweet,Sentiment
1,WinnersStrictly,1178090727709872128,2019-09-28 23:34:52,"[oakland, ca]","[ohio, state, over, team, total]",NaN,False,False,0.0
2,LucasLee45,1178090728418725889,2019-09-28 23:34:52,"[the, great, state, of, colorado]","[go, big, red, #huskers, #gbr]",NaN,False,False,0.0
8,PhilBergmanTV,1178090742197166080,2019-09-28 23:34:55,"[omaha, ne]","[brock, osweiler, at, today, s, #huskers, game]",NaN,False,False,-0.4
9,Prodby2LoKi,1178090747209359361,2019-09-28 23:34:56,"[norfside, huntsville, al]","[shewrap, ohio, state, when, they, win, the, a...",NaN,False,False,0.8
11,DownWithDamon,1178090757892255744,2019-09-28 23:34:59,"[cleveland, oh]","[let, s, go, buckeyes]",NaN,False,False,0.0


In [253]:
# then renaming the user locations to simply the state of origin

def stateGetter(list1, list2, list3, dict1):
    if any(item in list1 for item in list2): # first checking the first list against the second
        list1 = list(set(list1)&set(list2)) # getting only the value that matches (the abbreviation)
        list1 = list1[0] # and getting only the value itself
        list1 = dict1[list1] #then, setting the value of the user location to the corresponding state
        return list1
    elif any (item in list1 for item in list3):
        list1 = list(set(list1)&set(list3)) # getting only the value that matches (the state name)
        list1 = list1[0] # and getting only the value itself
        return list1

finalTweetSet['User Location'] = finalTweetSet['User Location'].apply(stateGetter, args = (stateAbbreviations,states,abbrevToState,))

In [254]:
#capitalizing state names to make visualization in Tableau easier later

finalTweetSet['User Location'] = finalTweetSet['User Location'].str.title()

#checking that it worked...
finalTweetSet['User Location'].head()

1     California
2       Colorado
8       Nebraska
9        Alabama
11          Ohio
Name: User Location, dtype: object

Finally, I need to convert the time stamp from UTC to Central Time (the game was played in Nebraska), which helps coordinate Tweets with game events.

In [255]:
#getting it as a time object
g = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
finalTweetSet['Time'] = finalTweetSet['Time'].apply(g)

# then subtracting five hours, so it's in Central Time, not UTC

finalTweetSet['Time'] = finalTweetSet['Time']-timedelta(hours = 5)
finalTweetSet['Time'].head()

1    2019-09-28 18:34:52
2    2019-09-28 18:34:52
8    2019-09-28 18:34:55
9    2019-09-28 18:34:56
11   2019-09-28 18:34:59
Name: Time, dtype: datetime64[ns]

And checking my beautiful data set, and sending it to a .csv so I can do analysis in Tableau.

In [256]:
finalTweetSet.head(15)

,Username,Tweet ID,Time,User Location,Text,Tweet Geo Coordinates,Is Retweet,Is Quote Tweet,Sentiment
1,WinnersStrictly,1178090727709872128,2019-09-28 18:34:52,California,"[ohio, state, over, team, total]",NaN,False,False,0.000000
2,LucasLee45,1178090728418725889,2019-09-28 18:34:52,Colorado,"[go, big, red, #huskers, #gbr]",NaN,False,False,0.000000
8,PhilBergmanTV,1178090742197166080,2019-09-28 18:34:55,Nebraska,"[brock, osweiler, at, today, s, #huskers, game]",NaN,False,False,-0.400000
9,Prodby2LoKi,1178090747209359361,2019-09-28 18:34:56,Alabama,"[shewrap, ohio, state, when, they, win, the, a...",NaN,False,False,0.800000
11,DownWithDamon,1178090757892255744,2019-09-28 18:34:59,Ohio,"[let, s, go, buckeyes]",NaN,False,False,0.000000
14,eyeformation,1178090766155046914,2019-09-28 18:35:01,Nebraska,"[i, think, tonight, is, the, night, that, adri...",NaN,False,False,0.000000
15,nutMEG_4,1178090765932781569,2019-09-28 18:35:01,Nebraska,"[hours, later, off, hours, of, sleep, but, it,...",NaN,False,False,0.400000
16,Broncos_Reddit,1178090768940048385,2019-09-28 18:35:01,Colorado,"[old, friend]",NaN,False,True,0.100000
19,jbc_sports_ops,1178090772127768579,2019-09-28 18:35:02,Georgia,"[clemson, survives, unc, but, what, a, game, n...",NaN,False,False,0.311111
25,intltrends,1178090776225570816,2019-09-28 18:35:03,Indiana,"[no, ohio, state, marching, to, victory, go, b...",NaN,False,False,0.000000


In [257]:
finalTweetSet.to_csv('Documents/Husker Project/cleanedGameTweets2.csv')